In [ ]:
import os
import sys
sys.path.append("/mnt/code/")

In [ ]:
import torchmetrics as tm

In [ ]:
import os
import tempfile
import time

import mlflow
from ray import air, tune
from ray.air import session
from domino_mlflow_utils.mlflow_callback import *

In [ ]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def train_function(config):
    width, height = config["width"], config["height"]

    for step in range(config.get("steps", 100)):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back to Tune.
        session.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)

In [ ]:
def tune_with_callback(mlflow_tracking_uri, experiment_name,finish_fast=False):
    cb = MyMLflowLoggerCallback(
                    tracking_uri=mlflow_tracking_uri,
                    experiment_name=experiment_name,
                    save_artifact=True,
                )
    tuner = tune.Tuner(
        train_function,
        tune_config=tune.TuneConfig(num_samples=5),        
        run_config=air.RunConfig(
            name="mlflow",
            callbacks=[
                cb
            ],
        ),
        param_space={
            "width": tune.randint(10, 100),
            "height": tune.randint(0, 100),
            "steps": 5 if finish_fast else 100,
        },
    )
    results = tuner.fit()
    cb.log_end_parent_run()

In [ ]:
import mlflow
import os
import ray
import logging
from domino_mlflow_utils.mlflow_callback import *

temp_dir='/tmp'

if not ray.is_initialized():
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    address=f"ray://{service_host}:{service_port}"
    #temp_dir='/mnt/data//{}/'.format(os.environ['DOMINO_PROJECT_NAME']) #set to a dataset
    ray.init(address=address, _temp_dir=temp_dir,runtime_env={"py_modules": ['/mnt/code/domino_mlflow_utils']})

print('Ray Initializied')
print(f'Ray Host={service_host} and Ray Port={service_port}')

experiment_name = 'RAYv7'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
tune_with_callback(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)

#tune_with_setup(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)
df = mlflow.search_runs(
        [mlflow.get_experiment_by_name(experiment_name).experiment_id]
    )
print(df)


In [ ]:
print(df.to_markdown())

|    | run_id                           |   experiment_id | status   | artifact_uri                                                        | start_time                       | end_time                         |   metrics.config/height |   metrics.mean_loss |   metrics.timestamp |   metrics.iterations |   metrics.pid |   metrics.config/steps |   metrics.training_iteration |   metrics.config/width |   metrics.time_since_restore |   metrics.time_this_iter_s |   metrics.time_total_s |   metrics.iterations_since_restore |   metrics.done |   metrics.trial_id |   params.height |   params.steps |   params.width | tags.mlflow.domino.hardware_tier   | tags.mlflow.runName        | tags.mlflow.source.type   | tags.mlflow.domino.data_sources   | tags.mlflow.domino.dataset_info                   | tags.mlflow.domino.user   | tags.mlflow.domino.run_id   | tags.mlflow.domino.project_name   | tags.mlflow.domino.project_id   | tags.mlflow.domino.environment_id   | tags.trial_name            |   tags.mlflow.domino.run_number | tags.mlflow.parentRunId          | tags.mlflow.user   | tags.mlflow.domino.root_run_id   | tags.mlflow.domino.user_id   | tags.mlflow.domino.environment_revision_id   | tags.mlflow.domino.provenance_checkpoint_id   | tags.mlflow.source.git.branch   | tags.mlflow.source.git.commit            |
|---:|:---------------------------------|----------------:|:---------|:--------------------------------------------------------------------|:---------------------------------|:---------------------------------|------------------------:|--------------------:|--------------------:|---------------------:|--------------:|-----------------------:|-----------------------------:|-----------------------:|-----------------------------:|---------------------------:|-----------------------:|-----------------------------------:|---------------:|-------------------:|----------------:|---------------:|---------------:|:-----------------------------------|:---------------------------|:--------------------------|:----------------------------------|:--------------------------------------------------|:--------------------------|:----------------------------|:----------------------------------|:--------------------------------|:------------------------------------|:---------------------------|--------------------------------:|:---------------------------------|:-------------------|:---------------------------------|:-----------------------------|:---------------------------------------------|:----------------------------------------------|:--------------------------------|:-----------------------------------------|
|  0 | ac9d9a22d17c443ba91aff6cb9464c48 |               1 | FINISHED | mlflow-artifacts:/mlflow/ac9d9a22d17c443ba91aff6cb9464c48/artifacts | 2024-03-05 03:39:57.729000+00:00 | 2024-03-05 03:40:06.093000+00:00 |                      32 |             3.5876  |         1.70961e+09 |                    4 |           781 |                      5 |                            5 |                     62 |                     0.40337  |                   0.100799 |               0.40337  |                                  5 |              0 |       nan          |              32 |              5 |             62 | small-k8s                          | train_function_fc644_00001 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e68e58e9f4eb77a8430762    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_fc644_00001 |                               4 | 3d690f5b48174b9195a50e9a76171cc0 | integration-test   | 3d690f5b48174b9195a50e9a76171cc0 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     |                                               |                                 |                                          |
|  1 | 8ccab96cdb704436a6fbcf9fd146077f |               1 | FINISHED | mlflow-artifacts:/mlflow/8ccab96cdb704436a6fbcf9fd146077f/artifacts | 2024-03-05 03:39:42.317000+00:00 | 2024-03-05 03:39:52.257000+00:00 |                      68 |             7.04631 |         1.70961e+09 |                    4 |           781 |                      5 |                            5 |                     99 |                     0.40359  |                   0.100848 |               0.40359  |                                  5 |              0 |       nan          |              68 |              5 |             99 | small-k8s                          | train_function_fc644_00000 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e68e58e9f4eb77a8430762    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_fc644_00000 |                               4 | 3d690f5b48174b9195a50e9a76171cc0 | integration-test   | 3d690f5b48174b9195a50e9a76171cc0 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     |                                               |                                 |                                          |
|  2 | 5c25bb2084964eabba3c20bfef97a126 |               1 | FINISHED | mlflow-artifacts:/mlflow/5c25bb2084964eabba3c20bfef97a126/artifacts | 2024-03-05 03:39:42.007000+00:00 | 2024-03-05 03:39:56.034000+00:00 |                      35 |             4.0618  |         1.70961e+09 |                    4 |           755 |                      5 |                            5 |                     42 |                     0.403657 |                   0.100684 |               0.403657 |                                  5 |              0 |       nan          |              35 |              5 |             42 | small-k8s                          | train_function_fc644_00003 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e68e58e9f4eb77a8430762    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_fc644_00003 |                               4 | 3d690f5b48174b9195a50e9a76171cc0 | integration-test   | 3d690f5b48174b9195a50e9a76171cc0 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     |                                               |                                 |                                          |
|  3 | 1c22a301ffa143fcb165f049f6374d36 |               1 | FINISHED | mlflow-artifacts:/mlflow/1c22a301ffa143fcb165f049f6374d36/artifacts | 2024-03-05 03:39:40.875000+00:00 | 2024-03-05 03:40:00.227000+00:00 |                      76 |             8.26667 |         1.70961e+09 |                    4 |           479 |                      5 |                            5 |                     35 |                     0.403359 |                   0.100827 |               0.403359 |                                  5 |              0 |       nan          |              76 |              5 |             35 | small-k8s                          | train_function_fc644_00004 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e68e58e9f4eb77a8430762    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_fc644_00004 |                               4 | 3d690f5b48174b9195a50e9a76171cc0 | integration-test   | 3d690f5b48174b9195a50e9a76171cc0 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     |                                               |                                 |                                          |
|  4 | b7f6cb6ecfe84d9a906c0ad55da9d0d0 |               1 | FINISHED | mlflow-artifacts:/mlflow/b7f6cb6ecfe84d9a906c0ad55da9d0d0/artifacts | 2024-03-05 03:39:40.567000+00:00 | 2024-03-05 03:40:00.810000+00:00 |                       8 |             1.29505 |         1.70961e+09 |                    4 |          2055 |                      5 |                            5 |                     48 |                     0.403402 |                   0.100738 |               0.403402 |                                  5 |              0 |       nan          |               8 |              5 |             48 | small-k8s                          | train_function_fc644_00002 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e68e58e9f4eb77a8430762    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_fc644_00002 |                               4 | 3d690f5b48174b9195a50e9a76171cc0 | integration-test   | 3d690f5b48174b9195a50e9a76171cc0 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     |                                               |                                 |                                          |
|  5 | 3d690f5b48174b9195a50e9a76171cc0 |               1 | RUNNING  | mlflow-artifacts:/mlflow/3d690f5b48174b9195a50e9a76171cc0/artifacts | 2024-03-05 03:39:37.717000+00:00 | NaT                              |                     nan |           nan       |       nan           |                  nan |           nan |                    nan |                          nan |                    nan |                   nan        |                 nan        |             nan        |                                nan |            nan |       nan          |                 |                |                | small-k8s                          | root-2024-03-05 03:39:38   | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e68e58e9f4eb77a8430762    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            |                            |                               4 |                                  | integration-test   |                                  | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     |                                               |                                 |                                          |
|  6 | 959b0702d493422db4c5afefcf8f0088 |               1 | FINISHED | mlflow-artifacts:/mlflow/959b0702d493422db4c5afefcf8f0088/artifacts | 2024-03-04 13:53:25.749000+00:00 | 2024-03-04 13:53:35.586000+00:00 |                      91 |             9.62632 |         1.70956e+09 |                    4 |          1200 |                      5 |                            5 |                     45 |                     0.403222 |                   0.100542 |               0.403222 |                                  5 |              0 |       nan          |              91 |              5 |             45 | small-k8s                          | train_function_8941d_00003 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_8941d_00003 |                               3 | fdeaaae8a3bd41f4be10db91f3ba3531 | integration-test   | fdeaaae8a3bd41f4be10db91f3ba3531 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
|  7 | 008d397ba3274bf29fe261ad9c2da268 |               1 | FINISHED | mlflow-artifacts:/mlflow/008d397ba3274bf29fe261ad9c2da268/artifacts | 2024-03-04 13:53:25.452000+00:00 | 2024-03-04 13:53:40.416000+00:00 |                      13 |             1.57933 |         1.70956e+09 |                    4 |           214 |                      5 |                            5 |                     87 |                     0.403474 |                   0.100857 |               0.403474 |                                  5 |              0 |       nan          |              13 |              5 |             87 | small-k8s                          | train_function_8941d_00004 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_8941d_00004 |                               3 | fdeaaae8a3bd41f4be10db91f3ba3531 | integration-test   | fdeaaae8a3bd41f4be10db91f3ba3531 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
|  8 | 14b9bc26b3fb415c827c71bc90d61056 |               1 | FINISHED | mlflow-artifacts:/mlflow/14b9bc26b3fb415c827c71bc90d61056/artifacts | 2024-03-04 13:53:15.566000+00:00 | 2024-03-04 13:53:37.031000+00:00 |                      28 |             3.04876 |         1.70956e+09 |                    4 |           253 |                      5 |                            5 |                     98 |                     0.403278 |                   0.100359 |               0.403278 |                                  5 |              0 |       nan          |              28 |              5 |             98 | small-k8s                          | train_function_8941d_00001 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_8941d_00001 |                               3 | fdeaaae8a3bd41f4be10db91f3ba3531 | integration-test   | fdeaaae8a3bd41f4be10db91f3ba3531 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
|  9 | b15338cfeb2248f7b020867a3281ff5e |               1 | FINISHED | mlflow-artifacts:/mlflow/b15338cfeb2248f7b020867a3281ff5e/artifacts | 2024-03-04 13:53:15.263000+00:00 | 2024-03-04 13:53:32.281000+00:00 |                      49 |             5.62464 |         1.70956e+09 |                    4 |           255 |                      5 |                            5 |                     32 |                     0.403605 |                   0.100868 |               0.403605 |                                  5 |              0 |       nan          |              49 |              5 |             32 | small-k8s                          | train_function_8941d_00002 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_8941d_00002 |                               3 | fdeaaae8a3bd41f4be10db91f3ba3531 | integration-test   | fdeaaae8a3bd41f4be10db91f3ba3531 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 10 | 57cb5c19928d4408a3bdf43da855319f |               1 | FINISHED | mlflow-artifacts:/mlflow/57cb5c19928d4408a3bdf43da855319f/artifacts | 2024-03-04 13:53:14.869000+00:00 | 2024-03-04 13:53:25.386000+00:00 |                      16 |             1.88249 |         1.70956e+09 |                    4 |          1200 |                      5 |                            5 |                     86 |                     0.403312 |                   0.100436 |               0.403312 |                                  5 |              0 |       nan          |              16 |              5 |             86 | small-k8s                          | train_function_8941d_00000 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_8941d_00000 |                               3 | fdeaaae8a3bd41f4be10db91f3ba3531 | integration-test   | fdeaaae8a3bd41f4be10db91f3ba3531 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 11 | fdeaaae8a3bd41f4be10db91f3ba3531 |               1 | RUNNING  | mlflow-artifacts:/mlflow/fdeaaae8a3bd41f4be10db91f3ba3531/artifacts | 2024-03-04 13:53:13.039000+00:00 | NaT                              |                     nan |           nan       |       nan           |                  nan |           nan |                    nan |                          nan |                    nan |                   nan        |                 nan        |             nan        |                                nan |            nan |       nan          |                 |                |                | small-k8s                          | root-2024-03-04 13:53:13   | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            |                            |                               3 |                                  | integration-test   |                                  | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 12 | e455ce3041614e3ca9e6c5f0d9584bdf |               1 | FINISHED | mlflow-artifacts:/mlflow/e455ce3041614e3ca9e6c5f0d9584bdf/artifacts | 2024-03-04 13:51:35.494000+00:00 | 2024-03-04 13:51:48.789000+00:00 |                      27 |             3.07594 |         1.70956e+09 |                    4 |           214 |                      5 |                            5 |                     64 |                     0.403759 |                   0.10086  |               0.403759 |                                  5 |              0 |         4.5749e+09 |              27 |              5 |             64 | small-k8s                          | train_function_45749_00003 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_45749_00003 |                               3 | db5151f010814778b0c4d7216287aec6 | integration-test   | db5151f010814778b0c4d7216287aec6 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 13 | 4c6649cd5a5a481fa25920345bff6afb |               1 | FINISHED | mlflow-artifacts:/mlflow/4c6649cd5a5a481fa25920345bff6afb/artifacts | 2024-03-04 13:51:23.823000+00:00 | 2024-03-04 13:51:36.286000+00:00 |                      46 |             6.11515 |         1.70956e+09 |                    4 |           755 |                      5 |                            5 |                     14 |                     0.403681 |                   0.100779 |               0.403681 |                                  5 |              0 |         4.5749e+09 |              46 |              5 |             14 | small-k8s                          | train_function_45749_00001 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_45749_00001 |                               3 | db5151f010814778b0c4d7216287aec6 | integration-test   | db5151f010814778b0c4d7216287aec6 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 14 | 7f8bddf845b142ecac6f102077caa35e |               1 | FINISHED | mlflow-artifacts:/mlflow/7f8bddf845b142ecac6f102077caa35e/artifacts | 2024-03-04 13:51:23.511000+00:00 | 2024-03-04 13:51:49.304000+00:00 |                      16 |             1.86178 |         1.70956e+09 |                    4 |           214 |                      5 |                            5 |                     93 |                     0.403641 |                   0.100829 |               0.403641 |                                  5 |              0 |         4.5749e+09 |              16 |              5 |             93 | small-k8s                          | train_function_45749_00004 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_45749_00004 |                               3 | db5151f010814778b0c4d7216287aec6 | integration-test   | db5151f010814778b0c4d7216287aec6 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 15 | 169f077f5e3f4c4b8105fbaec89a5ed2 |               1 | FINISHED | mlflow-artifacts:/mlflow/169f077f5e3f4c4b8105fbaec89a5ed2/artifacts | 2024-03-04 13:51:23.193000+00:00 | 2024-03-04 13:51:34.526000+00:00 |                      16 |             3.11515 |         1.70956e+09 |                    4 |           214 |                      5 |                            5 |                     14 |                     0.403685 |                   0.100792 |               0.403685 |                                  5 |              0 |         4.5749e+09 |              16 |              5 |             14 | small-k8s                          | train_function_45749_00002 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_45749_00002 |                               3 | db5151f010814778b0c4d7216287aec6 | integration-test   | db5151f010814778b0c4d7216287aec6 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 16 | fc1658f206eb4f37bb6979fde315faa5 |               1 | FINISHED | mlflow-artifacts:/mlflow/fc1658f206eb4f37bb6979fde315faa5/artifacts | 2024-03-04 13:51:22.219000+00:00 | 2024-03-04 13:51:45.636000+00:00 |                      36 |             3.9876  |         1.70956e+09 |                    4 |           214 |                      5 |                            5 |                     62 |                     0.403576 |                   0.100812 |               0.403576 |                                  5 |              0 |         4.5749e+09 |              36 |              5 |             62 | small-k8s                          | train_function_45749_00000 | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            | train_function_45749_00000 |                               3 | db5151f010814778b0c4d7216287aec6 | integration-test   | db5151f010814778b0c4d7216287aec6 | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |
| 17 | db5151f010814778b0c4d7216287aec6 |               1 | RUNNING  | mlflow-artifacts:/mlflow/db5151f010814778b0c4d7216287aec6/artifacts | 2024-03-04 13:51:19.540000+00:00 | NaT                              |                     nan |           nan       |       nan           |                  nan |           nan |                    nan |                          nan |                    nan |                   nan        |                 nan        |             nan        |                                nan |            nan |       nan          |                 |                |                | small-k8s                          | root-2024-03-04 13:51:20   | NOTEBOOK                  | unknown                           | 65e59517e9f4eb77a8430705-65e59517e9f4eb77a8430704 | integration-test          | 65e5d185e9f4eb77a843074f    | mlflow-demos                      | 65e59516e9f4eb77a84306ff        | 65e5a8e1e9f4eb77a8430724            |                            |                               3 |                                  | integration-test   |                                  | 65e594c0e9f4eb77a84306f4     | 65e5b1d4e9f4eb77a8430733                     | 65e5d1a8898e5646b23f1bb9                      | main                            | 73c860c73cba2a7830b6b7033faf23dd8080cf1e |